#**Indicadores de calidad de datos públicos**

En el presente notebook se realiza lo siguiente:     
* Proceso de extracción de datos públicos relacionado con noticias, temperatura y tipo de cambio. Estos datos son relevantes para el caso: previsión de demanda y estrategia de precios para solucionar la falta de inventario.

* Indicadores de calidad relevantes para el negocio que permitan identificar si el conjunto de datos extraído es ideal para ser utilizado en una solución analítica en la empresa y justificación de la elección de cada uno.

In [ ]:
!pip install feedparser

In [ ]:
import requests
import json
import feedparser
import random
from datetime import datetime
import pandas as pd

##**1. Noticias**

###**Extracción**

El dataset contiene categorias de noticias relevantes para el caso:

* Crisis económica o inflación → Puede afectar el poder adquisitivo y reducir compras.

* Días festivos o eventos deportivos → Pueden aumentar ventas de ciertos productos (ej. snacks para ver partidos).

* Huelgas o problemas logísticos → Pueden afectar entregas y disponibilidad de productos.

El dataset de salida contiene:


* Fecha: fecha de la publicacion de la noticia
* Tema: titulo de la noticia
* Categoría: categoria de la noticia
* Enlace: enlace a la publicacion de la noticia



In [ ]:
# URLs de Google News RSS para cada categoría
feeds = {
    "Crisis Económica": "https://news.google.com/rss/search?q=crisis+económica+OR+inflación+OR+recesión+Perú&hl=es-419&gl=PE&ceid=PE:es-419",

    "Festivos y Eventos Deportivos": "https://news.google.com/rss/search?q=eventos+deportivos+OR+festivos+OR+feriados+Perú&hl=es-419&gl=PE&ceid=PE:es-419",

    "Huelgas y Problemas Logísticos": "https://news.google.com/rss/search?q=huelgas+OR+paro+OR+bloqueo+OR+logística+Perú&hl=es-419&gl=PE&ceid=PE:es-419",
}

# Lista para almacenar noticias
lista_noticias = []

# Función para obtener noticias de una categoría específica
def obtener_noticias(url, categoria):
    feed = feedparser.parse(url)
    noticias_categoria = []

    for entrada in feed.entries:
        fecha_publicacion = pd.to_datetime(entrada.published)
        if fecha_publicacion.year == 2025:  # Filtrar solo noticias de 2025
            noticias_categoria.append({
                "Fecha": fecha_publicacion,
                "Tema": entrada.title,
                "Categoría": categoria,
                "Enlace": entrada.link
            })
        if len(noticias_categoria) >= 15:  # Recoge un poco más para elegir al azar
            break

    # Seleccionar aleatoriamente 10 noticias de la categoría
    random.shuffle(noticias_categoria)
    lista_noticias.extend(noticias_categoria[:10])  # Agregar solo 10 noticias

# Obtener noticias de cada categoría
for categoria, url in feeds.items():
    obtener_noticias(url, categoria)

# Convertir a DataFrame
df_noticias = pd.DataFrame(lista_noticias)

# Mostrar las primeras filas del DataFrame
print(df_noticias)



                 Fecha                                               Tema  \
0  2025-03-23 07:00:00  ¿Cobre en crisis? Por qué el posible arancel d...   
1  2025-03-21 07:00:00  Perú goleó a Bolivia: ¿Nos va igual en la canc...   
2  2025-03-27 22:36:54  Frontera Perú-Bolivia sufre los efectos de cri...   
3  2025-03-24 07:00:00       Programas de reactivación económica - gob.pe   
4  2025-04-03 03:54:50  ¿Es la CTS un fondo para enfrentar crisis econ...   
5  2025-03-22 07:00:00  Ola migratoria de bolivianos a Perú cada vez m...   
6  2025-03-20 07:00:00  Crisis en Bolivia: recurren al Perú para inten...   
7  2025-01-02 08:00:00  BCR devuelve la inflación de Perú al rango met...   
8  2025-04-02 12:49:57  Miles de peruanos viajan a Bolivia a pesar de ...   
9  2025-03-28 00:15:23  HASTA 3 MILLONES DE BOLIVIANOS PODRÍA MIGRAR A...   
10 2025-01-01 08:00:00  ¡Guardas las fechas! Calendario deportivo de l...   
11 2025-02-28 16:56:59  Gobierno definió el Comité Organizador de los ...   

###**Indicadores de calidad**

**Justificación de los Indicadores de Calidad de Datos**

 1. **Porcentaje de Completitud**:  
 Este indicador mide el grado de completitud de los datos en las columnas clave (`Fecha`, `Tema`, `Categoría`, `Enlace`). Un valor cercano al 100% indica que la mayoría de los registros están completos, lo cual es crucial para el análisis posterior. Si hay datos faltantes en estas columnas clave, puede haber un impacto significativo en la validez de cualquier análisis o predicción que se realice con los datos.

 2. **Porcentaje de Duplicidad**:   
 Este indicador mide cuántos registros son duplicados, es decir, cuántas noticias tienen la misma combinación de `Fecha`, `Tema`, `Categoría` y `Enlace`. Los duplicados pueden generar sesgo en el análisis, ya que estaríamos contando la misma noticia varias veces. Un porcentaje bajo de duplicados es crucial para asegurar que cada noticia es única, lo cual mejora la calidad del análisis y las predicciones basadas en estos datos.

 3. **Porcentaje de Nulos (Fecha, Tema, Categoría, Enlace)**:  
 Este indicador calcula el porcentaje de valores nulos en las columnas clave. Los valores nulos indican registros incompletos que pueden afectar la interpretación del dataset. Un alto porcentaje de valores nulos sugiere que los datos no son confiables o que hay áreas en el proceso de recolección de datos que necesitan mejora. Es importante monitorear los nulos para tomar decisiones sobre cómo manejar los datos faltantes, como la imputación o eliminación.

4. **Porcentaje de Fechas Válidas**:  
 Este indicador verifica si los valores en la columna `Fecha` son fechas válidas. Si hay valores no válidos o incorrectos en esta columna, el análisis temporal de las noticias y su relación con otros eventos o tendencias sería inexacto. Asegurar que las fechas sean correctas es fundamental para la cronología de los eventos y el análisis de tendencias a lo largo del tiempo.

5. **Porcentaje de Enlaces Únicos**:  
 Este indicador mide cuántos enlaces (URLs) son únicos en el dataset. Si hay enlaces repetidos, significa que las noticias pueden ser duplicadas o que hay problemas en la recopilación de los datos. Un alto porcentaje de enlaces únicos garantiza que las noticias provienen de fuentes distintas, lo que aumenta la diversidad y calidad de la información.

6. **Porcentaje de Categorías Válidas**:  
Este indicador comprueba si los registros están clasificados correctamente en las categorías predefinidas (`Crisis Económica`, `Festivos y Eventos Deportivos`, `Huelgas y Problemas Logísticos`). Una clasificación incorrecta de las noticias puede afectar la segmentación y los análisis posteriores. Este indicador asegura que los datos estén bien categorizados, lo cual es importante para hacer análisis segmentados y obtener insights más precisos por cada tipo de noticia.


Estos indicadores proporcionan una visión integral de la calidad del dataset, asegurando que los datos sean útiles y fiables para realizar análisis o predicciones en el contexto del análisis de la demanda en retail.


In [ ]:
def calcular_metricas_calidad(df):
    # 1. Porcentaje de completitud (no valores nulos en columnas clave)
    completitud = (df.dropna(subset=['Fecha', 'Tema', 'Categoría', 'Enlace']).shape[0] / df.shape[0]) * 100

    # 2. Porcentaje de duplicidad (duplicados en las columnas clave)
    duplicidad = (df.duplicated(subset=['Fecha', 'Tema', 'Categoría', 'Enlace']).sum() / df.shape[0]) * 100

    # 3. Porcentaje de valores nulos (en columnas clave)
    nulos = df[['Fecha', 'Tema', 'Categoría', 'Enlace']].isnull().mean() * 100

    # 4. Consistencia de las fechas (deben ser fechas válidas)
    fechas_validas = df['Fecha'].apply(lambda x: isinstance(x, pd.Timestamp)).mean() * 100

    # 5. Unicidad de los enlaces (porcentaje de enlaces únicos)
    enlaces_unicos = (df['Enlace'].nunique() / df.shape[0]) * 100

    # 6. Validación de categorías (comprobamos si solo hay categorías válidas)
    categorias_validas = df['Categoría'].isin(['Crisis Económica', 'Festivos y Eventos Deportivos', 'Huelgas y Problemas Logísticos']).mean() * 100

    # Crear un diccionario con las métricas
    metrica_calidad = {
        "***************** METRICAS DE CALIDAD DATASET NOTICIAS **********"
        "\n1. Porcentaje de Completitud": completitud,
        "2. Porcentaje de Duplicidad": duplicidad,
        "3. Porcentaje de Nulos (Fecha, Tema, Categoría, Enlace)": nulos,
        "4. Porcentaje de Fechas Válidas": fechas_validas,
        "5. Porcentaje de Enlaces Únicos": enlaces_unicos,
        "6. Porcentaje de Categorías Válidas": categorias_validas
    }

    return metrica_calidad

# Llamar a la función para calcular las métricas de calidad de los datos
metricas = calcular_metricas_calidad(df_noticias)

# Imprimir las métricas de calidad
for metrica, valor in metricas.items():
    if isinstance(valor, dict):
        # Si el valor es un diccionario (como los valores nulos), recorrerlo
        print(f"{metrica}:")
        for col, val in valor.items():
            print(f"  - {col}: {val:.2f}%")
    else:
        # Si el valor no es un diccionario, mostrarlo normalmente
        if isinstance(valor, pd.Series):
            # Si el valor es una Serie (como los valores nulos), recorrerla
            print(f"{metrica}:")
            for col, val in valor.items():
                print(f"  - {col}: {val:.2f}%")
        else:
            # Si es un valor escalar, mostrarlo en el formato deseado
            print(f"{metrica}: {valor:.2f}%")


***************** METRICAS DE CALIDAD DATASET NOTICIAS **********
1. Porcentaje de Completitud: 100.00%
2. Porcentaje de Duplicidad: 0.00%
3. Porcentaje de Nulos (Fecha, Tema, Categoría, Enlace):
  - Fecha: 0.00%
  - Tema: 0.00%
  - Categoría: 0.00%
  - Enlace: 0.00%
4. Porcentaje de Fechas Válidas: 100.00%
5. Porcentaje de Enlaces Únicos: 100.00%
6. Porcentaje de Categorías Válidas: 100.00%


##**2. Temperatura**

###**Extracción**

El dataset contiene las ciudades más relevantes del país(Perú)

El dataset de salida contiene:

* Fecha: fecha de la consulta a la api
* Ciudad: nombre de la ciudad
* Temperatura: temperatura de la ciudad


In [ ]:
# Coordenadas de ciudades en Perú
ciudades = {
    "Lima": (-12.0464, -77.0428),
    "Arequipa": (-16.4090, -71.5375),
    "Cusco": (-13.5320, -71.9675),
    "Trujillo": (-8.1150, -79.0299),
    "Piura": (-5.1945, -80.6328),
    "Chiclayo": (-6.7714, -79.8409),
    "Iquitos": (-3.7437, -73.2516),
    "Tacna": (-18.0066, -70.2463),
    "Huancayo": (-12.0651, -75.2049),
    "Pucallpa": (-8.3791, -74.5539),
    "Juliaca": (-15.4997, -70.1333),
    "Ayacucho": (-13.1588, -74.2239),
    "Cajamarca": (-7.1617, -78.5126),
    "Huaraz": (-9.5300, -77.5300),
    "Tarapoto": (-6.4831, -76.3650),
    "Tumbes": (-3.5669, -80.4515),
    "Moquegua": (-17.1956, -70.9350),
    "Puerto Maldonado": (-12.5933, -69.1833),
    "Chimbote": (-9.0743, -78.5936),
    "Puno": (-15.8402, -70.0219)
}

# Lista para almacenar los datos
datos_clima = []

for ciudad, (lat, lon) in ciudades.items():
    # API de Open-Meteo para obtener el clima actual
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"

    try:
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()

            # Extraer la temperatura en °C
            temperatura = data.get("current_weather", {}).get("temperature", None)

            # Guardar en la lista
            datos_clima.append({
                "fecha": datetime.now().strftime("%Y-%m-%d"),
                "ciudad": ciudad,
                "temperatura": temperatura
            })
        else:
            print(f"❌ Error obteniendo datos de {ciudad}: {response.status_code}")
    except Exception as e:
        print(f"⚠️ Error en {ciudad}: {e}")

# Convertir a DataFrame
df_clima = pd.DataFrame(datos_clima)

# Mostrar el resultado
print(df_clima)



         fecha            ciudad  temperatura
0   2025-04-04              Lima         20.5
1   2025-04-04          Arequipa         15.2
2   2025-04-04             Cusco          9.6
3   2025-04-04          Trujillo         21.0
4   2025-04-04             Piura         26.2
5   2025-04-04          Chiclayo         22.5
6   2025-04-04           Iquitos         25.5
7   2025-04-04             Tacna         16.8
8   2025-04-04          Huancayo         10.2
9   2025-04-04          Pucallpa         24.1
10  2025-04-04           Juliaca          6.8
11  2025-04-04          Ayacucho         14.4
12  2025-04-04         Cajamarca         13.2
13  2025-04-04            Huaraz         12.5
14  2025-04-04          Tarapoto         22.4
15  2025-04-04            Tumbes         26.0
16  2025-04-04          Moquegua         17.1
17  2025-04-04  Puerto Maldonado         24.9
18  2025-04-04          Chimbote         21.2
19  2025-04-04              Puno          9.8


**Justificación de los Indicadores de Calidad para el Dataset de Temperaturas**

1. **Porcentaje de Completitud**  
Este indicador mide el porcentaje de registros que no tienen valores nulos en las columnas clave del dataset (`fecha`, `ciudad`, `temperatura`). Es importante asegurarse de que los datos estén completos para garantizar que el análisis de las temperaturas no se vea afectado por datos faltantes, lo que podría conducir a conclusiones erróneas.

2. **Porcentaje de Duplicidad**  
El porcentaje de duplicidad muestra la proporción de filas duplicadas en el dataset, considerando las columnas clave. Los datos duplicados pueden distorsionar los análisis, ya que podrían hacer que ciertas ciudades o temperaturas aparezcan más veces de lo que realmente ocurren. Este indicador nos permite asegurarnos de que los datos sean únicos y representen adecuadamente los registros.

3. **Porcentaje de Valores Nulos**  
Este indicador calcula el porcentaje de valores nulos en las columnas clave (`fecha`, `ciudad`, `temperatura`). Un alto porcentaje de valores nulos puede indicar que la recolección de datos no fue adecuada o que los registros incompletos deben ser manejados (por ejemplo, mediante interpolación o eliminación de filas). Asegurarse de que los datos no tengan valores nulos es crucial para evitar errores en cualquier análisis posterior.

4. **Porcentaje de Fechas Válidas**  
Este indicador verifica que las fechas en la columna `fecha` sean válidas es esencial para la correcta interpretación de los datos. Las fechas mal formateadas o incorrectas podrían llevar a errores al analizar las tendencias temporales de las temperaturas. Este indicador asegura que todas las fechas sean procesables y útiles para análisis basados en el tiempo.

5. **Porcentaje de Ciudades Únicas**  
Este indicador mide el porcentaje de ciudades únicas en el dataset. En datasets de este tipo, las ciudades deben ser identificadas de manera única para evitar errores en la agregación de datos por ciudad. Un alto porcentaje de ciudades únicas sugiere que no hay redundancia en los registros y que las ciudades están representadas correctamente.

6. **Porcentaje de Temperaturas Fuera de Rango**  
Este indicador evalúa el porcentaje de registros cuya temperatura está fuera del rango mínimo(-13°C) y máximo(+40°C) del dataset. Las temperaturas fuera de este rango podrían indicar errores en la recolección de datos, o podrían representar situaciones inusuales que deben ser analizadas con mayor detalle. Identificar y excluir valores fuera de rango es importante para garantizar que las conclusiones sobre el clima sean precisas y confiables.


###**Indicadores de calidad**

In [ ]:
import pandas as pd

def calcular_metricas_clima(df):
    # 1. Porcentaje de Completitud (no valores nulos en columnas clave)
    completitud = (df.dropna(subset=['fecha', 'ciudad', 'temperatura']).shape[0] / df.shape[0]) * 100

    # 2. Porcentaje de duplicidad (duplicados en las columnas clave)
    duplicidad = (df.duplicated(subset=['fecha', 'ciudad', 'temperatura']).sum() / df.shape[0]) * 100

    # 3. Porcentaje de valores nulos (en columnas clave)
    nulos = df[['fecha', 'ciudad', 'temperatura']].isnull().mean() * 100

    # 4. Consistencia de las fechas (deben ser fechas válidas)
    fechas_validas = df['fecha'].apply(lambda x: isinstance(pd.to_datetime(x), pd.Timestamp)).mean() * 100

    # 5. Unicidad de las ciudades (porcentaje de ciudades únicas)
    ciudades_unicas = (df['ciudad'].nunique() / df.shape[0]) * 100

    # 6. Porcentaje de temperaturas fuera del rango (mínimo y máximo)
    temp_min = -13
    temp_max = 40
    fuera_de_rango = df[(df['temperatura'] < temp_min) | (df['temperatura'] > temp_max)]
    porcentaje_fuera_de_rango = (len(fuera_de_rango) / len(df)) * 100

    # Crear un diccionario con las métricas
    metrica_calidad = {
        "************** METRICAS DE CALIDAD DATASET TEMPERATURA ***************"
        "\n1. Porcentaje de Completitud": completitud,
        "2. Porcentaje de Duplicidad": duplicidad,
        "3. Porcentaje de Nulos (fecha, ciudad, temperatura)": nulos,
        "4. Porcentaje de Fechas Válidas": fechas_validas,
        "5. Porcentaje de Ciudades Únicas": ciudades_unicas,
        "6. Porcentaje de Temperaturas Fuera de Rango": porcentaje_fuera_de_rango
    }

    return metrica_calidad

# Llamar a la función para calcular las métricas de calidad de los datos
metricas_clima = calcular_metricas_clima(df_clima)

# Imprimir las métricas de calidad
for metrica, valor in metricas_clima.items():
    if isinstance(valor, dict):
        # Si el valor es un diccionario (como los valores nulos), recorrerlo
        print(f"{metrica}:")
        for col, val in valor.items():
            print(f"  - {col}: {val:.2f}%")
    else:
        # Si el valor no es un diccionario, mostrarlo normalmente
        if isinstance(valor, pd.Series):
            # Si el valor es una Serie (como los valores nulos), recorrerla
            print(f"{metrica}:")
            for col, val in valor.items():
                print(f"  - {col}: {val:.2f}%")
        else:
            # Si es un valor escalar, mostrarlo en el formato deseado
            print(f"{metrica}: {valor:.2f}%")


************** METRICAS DE CALIDAD DATASET TEMPERATURA ***************
1. Porcentaje de Completitud: 100.00%
2. Porcentaje de Duplicidad: 0.00%
3. Porcentaje de Nulos (fecha, ciudad, temperatura):
  - fecha: 0.00%
  - ciudad: 0.00%
  - temperatura: 0.00%
4. Porcentaje de Fechas Válidas: 100.00%
5. Porcentaje de Ciudades Únicas: 100.00%
6. Porcentaje de Temperaturas Fuera de Rango: 0.00%


##**3. Tipo de cambio**

###**Extracción**

El dataset contiene datos del tipo de cambio de soles a dolares en un determinada fecha

El dataset de salida contiene:


* Fecha: fecha de la consulta a la api del BCRP
* TC : tipo de cambio para la fecha dada

In [ ]:
# URL de la API del BCRP para obtener el tipo de cambio del dólar
url = "https://estadisticas.bcrp.gob.pe/estadisticas/series/api/PD04640PD/json"

# Realizar la solicitud a la API
response = requests.get(url)

# Verificar si la respuesta es exitosa
if response.status_code == 200:
    data = response.json()  # Convertir la respuesta a JSON

    # Extraer el tipo de cambio
    tipo_cambio = data["periods"]
    df_tipo_cambio = pd.DataFrame(tipo_cambio)

    # Renombrar columnas
    df_tipo_cambio.rename(columns={"name": "Fecha", "values": "TC"}, inplace=True)

    # Extraer el primer valor de cada lista en la columna "TC"
    df_tipo_cambio["TC"] = df_tipo_cambio["TC"].apply(lambda x: x[0])

    # Diccionario de meses en español
    meses_es = {
        "Ene": "Jan", "Feb": "Feb", "Mar": "Mar", "Abr": "Apr", "May": "May", "Jun": "Jun",
        "Jul": "Jul", "Ago": "Aug", "Sep": "Sep", "Oct": "Oct", "Nov": "Nov", "Dic": "Dec"
    }

    # Reemplazar los meses en español por inglés
    for mes_es, mes_en in meses_es.items():
        df_tipo_cambio["Fecha"] = df_tipo_cambio["Fecha"].str.replace(mes_es, mes_en)

    # Convertir a formato YYYY-MM-DD
    df_tipo_cambio["Fecha"] = pd.to_datetime(df_tipo_cambio["Fecha"], format="%d.%b.%y").dt.strftime("%Y-%m-%d")

    # Mostrar el DataFrame final
    print(df_tipo_cambio)
else:
    print("Error al obtener los datos de la API")


         Fecha     TC
0   2025-02-20  3.687
1   2025-02-21  3.683
2   2025-02-24  3.689
3   2025-02-25  3.686
4   2025-02-26  3.681
5   2025-02-27  3.674
6   2025-02-28  3.683
7   2025-03-03  3.691
8   2025-03-04  3.693
9   2025-03-05  3.663
10  2025-03-06  3.651
11  2025-03-07  3.659
12  2025-03-10  3.669
13  2025-03-11   3.67
14  2025-03-12  3.667
15  2025-03-13   3.67
16  2025-03-14  3.663
17  2025-03-17  3.655
18  2025-03-18  3.641
19  2025-03-19  3.626
20  2025-03-20  3.627
21  2025-03-21  3.641
22  2025-03-24  3.648
23  2025-03-25  3.644
24  2025-03-26  3.643
25  2025-03-27  3.649
26  2025-03-28  3.654
27  2025-03-31  3.677
28  2025-04-01  3.676
29  2025-04-02   3.68


###**Indicadores de calidad**

**Justificación de los Indicadores de Calidad para el Dataset de Tipo de Cambio (TC)**

1. **Porcentaje de Completitud**  
Este indicador mide el porcentaje de registros que no tienen valores nulos en las columnas clave del dataset (`Fecha`, `TC`). Es esencial para garantizar que todos los registros del tipo de cambio estén completos y que no haya datos faltantes que puedan afectar el análisis de la evolución del tipo de cambio.

2. **Porcentaje de Duplicidad**  
El porcentaje de duplicidad muestra la proporción de registros duplicados en el dataset, considerando las columnas clave. Los registros duplicados pueden distorsionar el análisis, haciendo que los valores del tipo de cambio aparezcan más veces de lo que realmente ocurrieron. Este indicador ayuda a asegurar que cada entrada sea única y representativa de una fecha específica.

3. **Porcentaje de Valores Nulos**  
Este indicador calcula el porcentaje de valores nulos en las columnas clave (`Fecha`, `TC`). Un alto porcentaje de valores nulos podría indicar problemas en la recolección o procesamiento de los datos, lo que podría impactar la precisión del análisis de las fluctuaciones del tipo de cambio. Se debe garantizar que no haya valores faltantes para asegurar la calidad de los datos.

4. **Porcentaje de Fechas Válidas**  
Este indicador verifica si las fechas en la columna `Fecha` son válidas y tienen un formato correcto. Las fechas mal formateadas o incorrectas pueden interferir con el análisis temporal de la serie del tipo de cambio. Asegurarse de que todas las fechas sean válidas es clave para cualquier análisis de tendencias a lo largo del tiempo.

5. **Porcentaje de Valores del Tipo de Cambio Fuera de Rango**  
Este indicador evalúa el porcentaje de registros donde el valor del tipo de cambio está fuera de un rango esperado basado en el mínimo(3.58) y máximo(4.32) de los registrados a lo largo del tiempo. Los valores fuera de este rango pueden indicar errores en los datos o fluctuaciones inusuales. Detectar estos valores es esencial para evitar que datos erróneos influyan en las conclusiones sobre la evolución del tipo de cambio.

6. **Porcentaje de Fechas Únicas**  
Este indicador mide el porcentaje de fechas únicas en el dataset. Es importante que cada fecha tenga un único tipo de cambio asociado para evitar inconsistencias en el análisis temporal. Un porcentaje bajo de fechas únicas podría sugerir duplicación de fechas o que los datos no están representando correctamente la evolución diaria del tipo de cambio.

7. **Porcentaje de Variación del Tipo de Cambio**  
Este indicador mide la variación promedio en el tipo de cambio a lo largo del tiempo, expresada en porcentaje. Ayuda a entender la volatilidad de la moneda, es decir, cómo fluctúan los valores del tipo de cambio a lo largo de los días. Un alto porcentaje de variación puede indicar mercados volátiles, lo cual es relevante para nuestro análisis.

8. **Porcentaje de Fechas en Feriados**  
Este indicador calcula el porcentaje de fechas en las que el tipo de cambio se registra en feriados (fechas no laborables). Los feriados suelen tener un comportamiento diferente en los mercados financieros, ya que las transacciones pueden ser limitadas. Es importante tener en cuenta estos días para ajustar los análisis de la evolución del tipo de cambio.

9. **Porcentaje de Registros en Fechas No Laborables**  
Este indicador calcula el porcentaje de registros del tipo de cambio que corresponden a fines de semana. Al igual que con los feriados, los fines de semana pueden presentar características especiales debido a la falta de actividad en los mercados. Este indicador es útil para ajustar los análisis que incluyen periodos de baja actividad comercial.


In [ ]:
def calcular_metricas_calidad_tipo_cambio(df):
    # Asegurarnos de que 'TC' esté como tipo numérico y 'Fecha' esté como tipo datetime
    df['TC'] = pd.to_numeric(df['TC'], errors='coerce')
    df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')

    # 1. Porcentaje de completitud (no valores nulos en columnas clave)
    completitud = (df.dropna(subset=['Fecha', 'TC']).shape[0] / df.shape[0]) * 100

    # 2. Porcentaje de duplicidad (duplicados en las columnas clave)
    duplicidad = (df.duplicated(subset=['Fecha', 'TC']).sum() / df.shape[0]) * 100

    # 3. Porcentaje de valores nulos (en columnas clave)
    nulos = df[['Fecha', 'TC']].isnull().mean() * 100

    # 4. Consistencia de las fechas (deben ser fechas válidas)
    fechas_validas = df['Fecha'].apply(lambda x: isinstance(x, pd.Timestamp)).mean() * 100

    # 5. Porcentaje de valores del tipo de cambio fuera de rango (fuera del mínimo y máximo del dataset)
    rango_min = 3.58
    rango_max = 4.32
    fuera_de_rango = ((df['TC'] < rango_min) | (df['TC'] > rango_max)).mean() * 100

    # 6. Porcentaje de fechas únicas (cada fecha debe tener un único valor de tipo de cambio)
    fechas_unicas = (df['Fecha'].nunique() / df.shape[0]) * 100

    # 7. Porcentaje de variación del tipo de cambio (volatilidad)
    variacion = df['TC'].pct_change().abs().mean() * 100  # Variación promedio en porcentaje

    # 8. Porcentaje de fechas en feriados (considerando días festivos como no laborables)
    # Suponiendo que tienes un listado de feriados (se necesita adaptar según el calendario)
    feriados = ['2025-01-01', '2025-05-01', '2025-06-03', '2025-07-28', '2025-07-29', '2025-08-30', '2025-10-08', '2025-11-01', '2025-12-08', '2025-12-25']
 # Ejemplo de feriados
    # Convertimos las fechas de feriados a formato datetime
    feriados = pd.to_datetime(feriados)
    fechas_feriados = df['Fecha'].isin(feriados).mean() * 100

    # 9. Porcentaje de registros en fechas no laborables (fines de semana)
    fechas_no_laborables = df['Fecha'].apply(lambda x: x.weekday() >= 5).mean() * 100

    # Crear un diccionario con las métricas
    metrica_calidad = {
        "************* METRICAS DE CALIDAD DATASET TC ********************"
        "\n1. Porcentaje de Completitud": completitud,
        "2. Porcentaje de Duplicidad": duplicidad,
        "3. Porcentaje de Nulos (Fecha, TC)": nulos,
        "4. Porcentaje de Fechas Válidas": fechas_validas,
        "5. Porcentaje de Valores Fuera de Rango (TC)": fuera_de_rango,
        "6. Porcentaje de Fechas Únicas": fechas_unicas,
        "7. Porcentaje de Variación del Tipo de Cambio (Volatilidad)": variacion,
        "8. Porcentaje de Fechas en Feriados": fechas_feriados,
        "9. Porcentaje de Registros en Fechas No Laborables": fechas_no_laborables
    }

    return metrica_calidad


# Llamar a la función para calcular las métricas
metricas = calcular_metricas_calidad_tipo_cambio(df_tipo_cambio)

# Imprimir las métricas de calidad
for metrica, valor in metricas.items():
    # Verificamos si el valor es un pd.Series (porcentaje de nulos o variación, que podrían ser Series)
    if isinstance(valor, pd.Series):
        print(f"{metrica}:")
        for col, val in valor.items():
            print(f"  - {col}: {val:.2f}%")
    else:
        # Si el valor no es una serie, lo imprimimos como valor escalar
        print(f"{metrica}: {valor:.2f}%")


************* METRICAS DE CALIDAD DATASET TC ********************
1. Porcentaje de Completitud: 100.00%
2. Porcentaje de Duplicidad: 0.00%
3. Porcentaje de Nulos (Fecha, TC):
  - Fecha: 0.00%
  - TC: 0.00%
4. Porcentaje de Fechas Válidas: 100.00%
5. Porcentaje de Valores Fuera de Rango (TC): 0.00%
6. Porcentaje de Fechas Únicas: 100.00%
7. Porcentaje de Variación del Tipo de Cambio (Volatilidad): 0.21%
8. Porcentaje de Fechas en Feriados: 0.00%
9. Porcentaje de Registros en Fechas No Laborables: 0.00%
